In [4]:
import json
import pandas as pd
import re
from collections import Counter
import os
from tqdm import tqdm

In [6]:
'''
Counts pronouns from one tweet text
returns a dictionary of the prounouns with its count value
'''
def count_pronouns_for1JSON(one_tweet_text):
    hen_patter = re.compile("[^a-z]hen[^a-z]")
    han_patter = re.compile("[^a-z]han[^a-z]")
    hon_patter = re.compile("[^a-z]hon[^a-z]")
    den_patter = re.compile("[^a-z]den[^a-z]")
    det_patter = re.compile("[^a-z]det[^a-z]")
    denna_patter = re.compile("[^a-z]denna[^a-z]")
    denne_patter = re.compile("[^a-z]denne[^a-z]")
    counters = {'hen':0,'han':0,'hon':0,'den':0,'det':0,'denna':0,'denne':0}
       # Hen Patter     
    if hen_patter.search(one_tweet_text):
        counters['hen'] += 1
    # “han” Patter       
    if han_patter.search(one_tweet_text):
        counters['han'] += 1
    # HON        
    if hon_patter.search(one_tweet_text):
         counters['hon'] += 1
           # DET        
    if den_patter.search(one_tweet_text):
        counters['den'] += 1
        # DET        
    if det_patter.search(one_tweet_text):
         counters['det'] += 1
    # DENNA        
    if denna_patter.search(one_tweet_text):
         counters['denna'] += 1
    # DENNE       
    if denne_patter.search(one_tweet_text):
        counters['denne'] += 1
    return counters

In [7]:
'''
LOCAL TEST
Read through all tweet-files and return the final results in a dictionary
'''
%%time
results ={}
counter = 0
# input_path = "./data/05cb5036-2170-401b-947d-68f9191b21c6"
#input_path = 'untitled.txt'
input_path = "./data"
for filename in tqdm(os.listdir(input_path)):
    with open(os.path.join(input_path, filename)) as file:
        for jsonObject in file:
            if not jsonObject.isspace():
                json_tweet = json.loads(jsonObject)  
                if (json_tweet['retweeted'] == False): 
                    text = json_tweet['text'].lower()
                    result = count_pronouns_for1JSON(text)
                    counter = counter +  1
            results = Counter(results) + Counter(result)

print(dict(results))
print(counter)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [01:03<00:00,  1.59it/s]

{'hon': 80306, 'han': 237298, 'det': 180874, 'den': 485848, 'hen': 8012, 'denne': 2600, 'denna': 7980}
452723
CPU times: user 55.1 s, sys: 2.45 s, total: 57.5 s
Wall time: 1min 3s


In [8]:
#--------------------------------CELERY----------------------------------------------------------------------------------------
#First: Queue all Tasks first by counting the pronouns
#Second: Loop throug all tasks object ASYNCRESULT, call get() and sum all results into a dictionary

In [9]:
%%time
# Celery task
from proj.tasks import addDict, mul, count_pronouns_for1JSON
results ={}
queuedTask =[]
#input_path = "./data/05cb5036-2170-401b-947d-68f9191b21c6"
#input_path = 'untitled.txt'
input_path = "./data"
for filename in tqdm(os.listdir(input_path)): #(os.path.join(input_path, filename)
    with open(os.path.join(input_path, filename)) as file:
        for jsonObject in file:
            if not jsonObject.isspace():
                json_tweet = json.loads(jsonObject)  
                if (json_tweet['retweeted'] == False): 
                    text = json_tweet['text'].lower()
                    queuedTask.append(count_pronouns_for1JSON.delay(text))


        
        
queuedTask


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [35:29<00:00, 21.08s/it]

CPU times: user 7min 12s, sys: 34.6 s, total: 7min 47s
Wall time: 35min 29s


[<AsyncResult: 01c2c734-729c-4a52-85ff-9b87818caef1>,
 <AsyncResult: bcf24e51-c5e6-4ac0-9f8c-c2c83a8016e5>,
 <AsyncResult: 8ee1a79d-e414-4567-b078-d009c3e81f05>,
 <AsyncResult: 2a40796a-293a-435e-b8b0-b1b52376f3c1>,
 <AsyncResult: d0df6b31-e473-47bd-936e-35955b628d79>,
 <AsyncResult: 5d3766b8-7ffd-4734-be37-59612da1a2da>,
 <AsyncResult: 6fe84c7c-96fa-4454-93d5-59c56ab4c3dc>,
 <AsyncResult: b9db71df-4d22-48e4-a75f-730fad22acdb>,
 <AsyncResult: c68df5da-6374-4216-94df-a7fbc038e540>,
 <AsyncResult: 6b50e836-6281-4d7e-bb25-9228051e4703>,
 <AsyncResult: f1d454ed-55de-4735-84d8-03f90f4ca91b>,
 <AsyncResult: a99a15ff-f50d-4ae8-b8fb-016b183195a1>,
 <AsyncResult: 6d42fefb-49a7-41ea-9b33-72ede100d5b5>,
 <AsyncResult: 3499282c-5f5d-41c6-a11e-615e8de617f5>,
 <AsyncResult: b0ec6535-ec23-4d69-b0ad-44f73cb7aa66>,
 <AsyncResult: 3db4b1af-a9e2-4645-85d2-38ba9a12c6b3>,
 <AsyncResult: 9375016c-899b-41d7-ac85-a6d4d2dae8a4>,
 <AsyncResult: 8134d909-ace2-4b85-afa3-f1a77f800377>,
 <AsyncResult: 73997454-34a1

In [10]:
len(queuedTask)

452723

In [ ]:
%%time
results = queuedTask[0].get()
for i in range(1,len(queuedTask)):
    results = Counter(results) + Counter(queuedTask[i].get())
    
results
    

In [1]:
#-----------------------WEAK SCALING TEST-------------------------------------------------
#### 

In [7]:
def count_pronouns():
    # Celery task
    from proj.tasks import count_pronouns_for1JSON
    queuedTask =[]
    results = {}
    input_path = ["./data/3c0e60ce-1f50-496a-8335-7a118f50843c",
                  "./data/332a01e3-f9f7-4d5f-bc9c-19a6263cb620",
                  "./data/8474d838-31d6-43e4-a437-282612c3fd53",
                 "./data/1e73ea44-9842-4aa3-b043-603ad3edcc1d",
                 "./data/c7f4aa96-1b2f-4ced-a265-cef17de64f8b",]
    #input_path = 'untitled.txt'
    for path in input_path:
        with open(path) as file:
            for jsonObject in file:
                if not jsonObject.isspace():
                    json_tweet = json.loads(jsonObject)  
                    if (json_tweet['retweeted'] == False): 
                        text = json_tweet['text'].lower()
                        queuedTask.append(count_pronouns_for1JSON.delay(text))

    results = queuedTask[0].get()
    for i in range(1,len(queuedTask)):
        results = Counter(results) + Counter(queuedTask[i].get())

    return results

In [ ]:
%%time
print(count_pronouns())

In [ ]:
# %%time
# results = {}
# input_path = "./data"
# for filename in os.listdir(input_path):
#     tweet_data_frame = read_input(os.path.join(input_path, filename))
#     for index, line in tweet_data_frame.items():
#         result_dic = count_pronouns(tweet_data_frame['text'])
#         results = Counter(results) + Counter(result_dic)
#results